### make table + mini eda S1 - list of terms used in data + date ranges
* Also of reviews amount!

In [1]:
import pandas as pd

In [2]:
TREND_FILE = "trends_v6.csv.gz"

In [3]:
df = pd.read_csv(TREND_FILE,parse_dates=["Year"],infer_datetime_format=True) # ,"first","start"
#### NEW! Sort by date!
df = df.sort_values(["Year","variable"]).reset_index(drop=True)
df = df[['variable', 'y_raw',
       'first', 'start', 'year_num',
        "reviews",#"research_review_diff",
         "research_review_ratio"]]
## y_raw is research and reviews together
df["reviews_div_research"] = df["reviews"].div(df["y_raw"].sub(df["reviews"]))
df["reviews_ratio"] = df["reviews"].div(df["y_raw"])
# df["first"] = pd.to_datetime(df["first"])#.dt.year
df["first"] = df["first"].str.split("-",expand=True)[0].astype(int)
df["start"] = pd.to_datetime(df["start"]).dt.year.astype(int)

df["Years from First ever occurrence to (first valid) Start date"] = df["start"].sub(df["first"])
df["Years since Start date"] = df["year_num"].sub(df["start"])
df = df.round(3)
df

,variable,y_raw,first,start,year_num,reviews,research_review_ratio,reviews_div_research,reviews_ratio,Years from First ever occurrence to (first valid) Start date,Years since Start date
0,BLAST,17.484,1945,1950,1950,0.000,1.000,0.000,0.000,5,0
1,EEG,346.186,1945,1950,1950,0.000,1.000,0.000,0.000,5,0
2,Epidemiology,416.123,1945,1950,1950,0.000,1.000,0.000,0.000,5,0
3,Histamine,635.257,1945,1950,1950,0.000,1.000,0.000,0.000,5,0
4,Medulla oblongata,19.815,1945,1950,1950,0.000,1.000,0.000,0.000,5,0
...,...,...,...,...,...,...,...,...,...,...,...
7028,vaccine,1641.145,1945,1950,2020,314.802,0.966,0.237,0.192,5,70
7029,viral therapy,1272.563,1946,1952,2020,267.545,0.976,0.266,0.210,6,68
7030,vitamin C,186.814,1945,1950,2020,16.183,0.983,0.095,0.087,5,70
7031,vitamin D,351.045,1945,1950,2020,54.087,0.977,0.182,0.154,5,70


In [4]:
df.groupby("variable")["year_num"].max().unique()

array([2020, 1991])

In [5]:
df = df.loc[(df["year_num"]> df["start"]) & (df["year_num"]>= 1979) ].reset_index(drop=True)

# if FILTER_GROUP_SIZES:
#     print(df.shape[0])
#     element_group_sizes = df['variable'].groupby(df['variable']).transform('count')
#     df = df_targets[df>6] # was 5, changed for CV results
#     df.reset_index(inplace=True,drop=True) # otherwise, error ?? 
#     print(df_targets.shape[0])

## how many years counts ofr each group (in valid time range):
df["# Years of Topic coverage in dataset (After dates filtering)"] = df['variable'].groupby(df['variable']).transform('count')
df["Average topic popularity value (After dates filtering)"] = df.groupby(df['variable'])["y_raw"].transform('mean')
df["Max topic popularity value (After dates filtering)"] = df.groupby(df['variable'])["y_raw"].transform('max')
df["Minimum topic popularity value (After dates filtering)"] = df.groupby(df['variable'])["y_raw"].transform('min')
df["Year of topics max popularity (After dates filtering)"] = df.loc[df.groupby(df['variable'])["y_raw"].transform("idxmax")]["year_num"].values
df["Year of topics min popularity (After dates filtering)"] = df.loc[df.groupby(df['variable'])["y_raw"].transform("idxmin")]["year_num"].values

df["Fraction of topic papers that were reviews in year of topics max popularity"] = df.loc[df.groupby(df['variable'])["y_raw"].transform("idxmax")]["reviews_ratio"].values
df["Fraction of topic papers that were reviews in year of topics lowest popularity"] = df.loc[df.groupby(df['variable'])["y_raw"].transform("idxmin")]["reviews_ratio"].values

df["Average ratio of reviews to overall topic popularity (After dates filtering)"] = df.groupby(df['variable'])["reviews_ratio"].transform('mean')
df["Average topic reviews popularity value (After dates filtering)"] = df.groupby(df['variable'])["reviews"].transform('mean')

df["Years from First ever occurrence to (first valid) Start date"] = df.groupby(df['variable'])["Years from First ever occurrence to (first valid) Start date"].transform('max')
df["Years since Start date"] = df.groupby(df['variable'])["Years since Start date"].transform('max')
df = df.round(2)
df = df.drop(columns=["y_raw","year_num",
                     "reviews","research_review_ratio","reviews_div_research","reviews_ratio"],).drop_duplicates().reset_index(drop=True)
df.rename(columns={"variable":"Topic","first":"First occurrence year","start":"First valid occurrence date"},inplace=True)
df

,Topic,First occurrence year,First valid occurrence date,Years from First ever occurrence to (first valid) Start date,Years since Start date,# Years of Topic coverage in dataset (After dates filtering),Average topic popularity value (After dates filtering),Max topic popularity value (After dates filtering),Minimum topic popularity value (After dates filtering),Year of topics max popularity (After dates filtering),Year of topics min popularity (After dates filtering),Fraction of topic papers that were reviews in year of topics max popularity,Fraction of topic papers that were reviews in year of topics lowest popularity,Average ratio of reviews to overall topic popularity (After dates filtering),Average topic reviews popularity value (After dates filtering)
0,BLAST,1945,1950,5,70,42,166.42,198.61,103.33,1989,1980,0.04,0.03,0.06,9.99
1,Brodmann area,1951,1978,27,42,42,7.75,15.28,0.71,2005,1980,0.02,0.00,0.04,0.31
2,Cingulate cortex,1946,1951,5,69,42,79.36,162.47,13.82,2013,1979,0.07,0.00,0.06,5.25
3,DNA array,1956,1978,22,42,42,335.22,991.05,4.25,2010,1979,0.08,0.08,0.09,31.59
4,Dorsolateral prefrontal cortex,1960,1975,15,45,42,29.16,68.94,0.36,2014,1980,0.05,0.00,0.06,1.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,miRNA,1970,2005,35,15,15,696.62,1258.89,94.80,2019,2006,0.12,0.30,0.18,112.38
124,CRISPR,2002,2007,5,13,13,134.06,385.56,3.34,2020,2008,0.15,0.18,0.15,21.17
125,biosimilar,1977,2009,32,11,11,30.77,53.36,7.75,2018,2010,0.23,0.37,0.28,8.24
126,connectome,2005,2013,8,7,7,78.92,104.56,47.29,2019,2014,0.08,0.16,0.11,8.03


In [6]:
# (df["Fraction of papers that were reviews in year of topics lowest popularity"]>df["Fraction of papers that were reviews in year of topics max popularity"]).mean()

KeyError: 'Fraction of papers that were reviews in year of topics lowest popularity'

In [ ]:
df.to_csv("S1-topics_list.csv",index=False)